# Experiment 1: Varying Randman

## Generate randman

In [ ]:
from src.RandmanFunctions import RandmanConfig, generate_and_save_randman

data_configs = [{'nb_classes': 2, 'nb_units': 3, 'alpha': 3, 'dim_manifold': 1},
                {'nb_classes': 5, 'nb_units': 3, 'alpha': 3, 'dim_manifold': 1},
                {'nb_classes': 2, 'nb_units': 10, 'alpha': 3, 'dim_manifold': 1},
                {'nb_classes': 2, 'nb_units': 3, 'alpha': 1, 'dim_manifold': 1},
                {'nb_classes': 2, 'nb_units': 3, 'alpha': 3, 'dim_manifold': 2}]

In [ ]:
for data_config in data_configs:
    generate_and_save_randman(RandmanConfig(**data_config))

## Generate problem

### BBOB problem
What BBOB problems should be generated? 
* 24 functions
* dims: 2, 160, 640 
* 3 instances 

In [ ]:
from src.LandscapeAnalysis import BBOBProblemConfig
def generate_bbob():
    # Generate all combinations of BBOB problems with the given configurations
    functions = list(range(1, 25))  # 24 functions
    dimensions = [2, 160, 640]      # 3 dimensions
    instances = [1, 2, 3]           # 3 instances

    for function in functions:
        for dim in dimensions:
            for instance in instances:
                problem_config = BBOBProblemConfig(function, instance, dim)
                problem_config.write_to_db(db_path='data/landscape-analysis.db')
generate_bbob()

### Generate samples and match with problems

How many samples should be used? Why not 30. 

In [ ]:
import sqlite3
from math import ceil, log2
from src.LandscapeAnalysis import ParameterSampleConfig, assign_samples_to_problem, BBOBProblemConfig

def assign_samples_to_bbob():
    # Connect to the SQLite database
    db_path = "data/landscape-analysis.db"
    con = sqlite3.connect(db_path)
    cur = con.cursor()

    cur.execute("SELECT id, dim FROM bbob_problems")
    rows = cur.fetchall()
    con.close()

    for problem_id, dim in rows:
        nb_samples = 2**(ceil(log2(50 * dim)))  # sobol's sample size must be a power of 2
        max_nb_versions = 30
        lower_bound = -5 # bbob are defined on [-5, 5]
        upper_bound = 5
        sample_config = ParameterSampleConfig(dim, nb_samples, "sobol", lower_bound, upper_bound)
        problem_config = BBOBProblemConfig.lookup_by_id(problem_id)
        assign_samples_to_problem(problem_config, sample_config, max_nb_versions, 'data/samples', 'data/landscape-analysis.db')
assign_samples_to_bbob()
  

## Calculate loss

In [ ]:
from src.LandscapeAnalysis import get_next_available_id, calculate_and_save_loss

while (id := get_next_available_id('loss_filename')) is not None:
    calculate_and_save_loss(id)

## Extract Features

In [ ]:
from pflacco.classical_ela_features import *
from src.LandscapeAnalysis import get_next_available_id, calculate_and_save_features
while (id := get_next_available_id('ic_h_max')) is not None:
    calculate_and_save_features(id, calculate_information_content)

Column 'ic_h_max' added to 'loss_surfaces' table by get_next_available_id()


In [ ]:
from pflacco.classical_ela_features import *
import numpy as np

def find_feature_name():
    from pflacco.sampling import create_initial_sample

    # Arbitrary objective function
    def objective_function(x):
        return np.random.rand()

    dim = 3
    # Create inital sample using latin hyper cube sampling
    X = create_initial_sample(dim, sample_type = 'lhs')
    # Calculate the objective values of the initial sample
    # using an arbitrary objective function
    y = X.apply(lambda x: objective_function(x), axis = 1)

    # Compute the 3 feature sets from the classical ELA features which are solely based on the initial sample
    ela_meta = calculate_ela_meta(X, y)
    print(ela_meta)
find_feature_name()

{'r20_ela_meta.lin_simple.adj_r2': -0.005409156310980379, 'r20_ela_meta.lin_simple.intercept': 0.4715494876893619, 'r20_ela_meta.lin_simple.coef.min': 0.0020344682591832736, 'r20_ela_meta.lin_simple.coef.max': 0.08213140207973536, 'r20_ela_meta.lin_simple.coef.max_by_min': 40.36995991901421, 'r20_ela_meta.lin_w_interact.adj_r2': -0.019009364039383136, 'r20_ela_meta.quad_simple.adj_r2': -0.0057795330142678125, 'r20_ela_meta.quad_simple.cond': 5.64899122378976, 'r20_ela_meta.quad_w_interact.adj_r2': 0.002901501956317998, 'r20_ela_meta.costs_runtime': 0.015}


# Experiment 2: Varying SNN

## Generate SNN models

In [2]:
from src.Models import RandmanSNNConfig

def generate_snn_models_configs():
    # constants in this experiment
    BETA = 0.95
    LEARN_BETA = True
    PARAMETER_TYPE = 'weights'
    
    # variables in this experiment
    nb_hidden_1_list = [100, 100, 30, 100]
    recurrent_list = [False, True, False, False]
    nb_hidden_2_list = [-1, -1, -1, 10]
    for i in range(len(nb_hidden_1_list)):
        config = RandmanSNNConfig(
            nb_hidden_1=nb_hidden_1_list[i],
            nb_hidden_2=nb_hidden_2_list[i],
            beta=BETA,
            learn_beta=LEARN_BETA,
            recurrent=recurrent_list[i],
            parameter_type=PARAMETER_TYPE
        )
        config.write_to_db(db_path='data/landscape-analysis.db')
generate_snn_models_configs()